<a href="https://colab.research.google.com/github/ros-lin/hf-llm-demo/blob/main/notebooks/hugging-face-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# simple installation
# !pip install transformers

# heavy installation
!pip install transformers[sentencepiece]

In [3]:
import transformers